In [ ]:
using LCIO
using Printf
using Corpuscles
using LinearAlgebra
using ProgressMeter
using Plots
using UnicodeFun

In [ ]:
flavorTagDir = readdir("/nfs/dust/ilc/group/ild/miniDST/E250-SetA/ILD/flavortag", join=true)
bbFiles = filter(name -> occursin(r"h_bb",name), flavorTagDir)
ccFiles = filter(name -> occursin(r"h_cc",name), flavorTagDir)
uuFiles = filter(name -> occursin(r"h_uu",name), flavorTagDir)
ddFiles = filter(name -> occursin(r"h_dd",name), flavorTagDir)
ggFiles = filter(name -> occursin(r"h_gg",name), flavorTagDir)
ssFiles = filter(name -> occursin(r"h_ss",name), flavorTagDir)

# First steps in LCIO
Simple notebook to get to know a new file
- Print the collections in the first event
- Print the MC Particle table for the first event
- Print a few simple statistics about the file (number of particles)

In [ ]:
# LCIO.open(bbFiles[1]) do reader
LCIO.open("../data/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I410001.Pn23n23h_bb.eL.pR.n001.d_dstm_14987_13.slcio") do reader

    for (iev, event) in enumerate(reader)
        println(repeat("/", 35))
        println("EVENT: ", getEventNumber(event))
        println("RUN: ", getRunNumber(event))
        println("DETECTOR: ", getDetectorName(event))
        println("COLLECTIONS: (see below)")
        println(repeat("/", 35))
        println()
        println(repeat("-", 100))
        @printf("%-40s%-40s%-10s\n", "Collection Name", "Element Type", "Elements")
        println(repeat("=", 100))
        for name in getCollectionNames(event)
            c = getCollection(event, name)
            @printf("%-40s%-40s%-10d\n", name, getTypeName(c), length(c))
        end
        println(repeat("-", 100))
        println()
        println()
        if iev > 2
            break
        end
    end
end

## Printing the list of MCParticles

In [ ]:
fileList_gg = filter(s->occursin(r"E250_SetA.Pmumuh2gg.Gwhizard-2_84.eL0.8\.pR0.3\..*_DST.slcio", s), readdir("/nfs/dust/ilc/user/jstrube/StrangeHiggs/data/RecoLevel", join=true))

LCIO.open(fileList_gg[1]) do reader
    for event in reader
        @printf("%-6s %10s %-15s%-10s%-16s%-3s\n", "Index", "PDG", "name", "parents", "momentum (GeV)", "generatorStatus")
        for (idx, mcp) in enumerate(getCollection(event, "MCParticle"))
            parentPDGs = Int[]
            pdg = getPDG(mcp)
            if 90 < pdg < 99 
                name = "internal"
            elseif abs(pdg) > 20000
                name = "unknown"
            else
                name = Particle(pdg).name
                parents = getParents(mcp)
                for p in parents
                    push!(parentPDGs, getPDG(p))
                end            
            end
            @printf("%6s %10s %-15s%-10s%13.3f%7s\n",
                idx,
                pdg,
                name,
                join(parentPDGs, " "),
                norm(getMomentum(mcp)),
                getGeneratorStatus(mcp)
            )
        end
        println(repeat("-", 100))
        break
    end
end

## Printing the PFO List

In [ ]:
LCIO.open("../data/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I410001.Pn23n23h_bb.eL.pR.n001.d_dstm_14987_13.slcio") do reader
    for (iEvent, event) in enumerate(reader)
        @printf("%-6s %10s %-15s%-10s%-16s%-3s\n", "Index", "PDG", "name", "parents", "momentum (GeV)", "generatorStatus")
        for (idx, pfo) in enumerate(getCollection(event, "PandoraPFOs"))
            @printf("%6s %10s %15s\n",
                idx,
                getType(pfo),
                Particle(getType(pfo)).name
            )
        end
        println(repeat("-", 100))
        for v0 in getCollection(event, "V0RecoParticles")
            @printf("V0: %s\n", getType(v0))
        end
        if iEvent > 4
            break
        end
    end
end

😞
OK, so we have to be sure to check some of the collections _before_ reading them. Not every event contains all collections... 

We can use the following to check for this.
```
        if "V0RecoParticles" ∉ getCollectionNames(event)
            continue
        end
```

## Making a histogram

In [ ]:
nFinalState = Int[]
LCIO.open("../data/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I410001.Pn23n23h_bb.eL.pR.n001.d_dstm_14987_13.slcio") do reader
    println(length(reader), " events in the file")
    @showprogress for event in reader
        finalStates = sum([1 for mcp in getCollection(event, "MCParticlesSkimmed") if getGeneratorStatus(mcp) == 1])
        push!(nFinalState, finalStates)
    end
end
histogram(nFinalState, xlabel="Number of FinalState MCParticles in an event")

## Accessing the PID info

In [ ]:
LCIO.open("../data/rv02-02.sv02-02.mILD_l5_o1_v02.E250-SetA.I410001.Pn23n23h_bb.eL.pR.n001.d_dstm_14987_13.slcio") do reader
    for event in reader
        pfos = getCollection(event, "PandoraPFOs")
        # create a PID handler to access th b-tag information
        pidh = PIDHandler(pfos)
        idList = LCIO.getAlgorithmIDs(pidh)
        for i in idList
            println(i, "\t", String(LCIO.getAlgorithmName(pidh, i)))
            for name in LCIO.getParameterNames(pidh, i)
                println("\t", name)
            end
        end
        break
    end
end

In [ ]:
# let's take a look at what PID algorithms are available on jets and print the values of lcfiplus on the two jets in the first event
LCIO.open(bbFiles[1]) do reader
    for event in reader
        jets = getCollection(event, "Refined2Jets")
        pidh = PIDHandler(jets)
        algoList = LCIO.getAlgorithmIDs(pidh)
        for i in algoList
            algoName = String(LCIO.getAlgorithmName(pidh, i))
            println(i, "\t", algoName)
            iAlgo = LCIO.getAlgorithmID(pidh, algoName)
            for name in LCIO.getParameterNames(pidh, iAlgo)
                println(name)
            end
        end
        ilcfi = LCIO.getAlgorithmID(pidh, "lcfiplus")
        ibtag = getParameterIndex(pidh, ilcfi, "BTag")
        ictag = getParameterIndex(pidh, ilcfi, "CTag")
        iotag = getParameterIndex(pidh, ilcfi, "OTag")
        icat = getParameterIndex(pidh, ilcfi, "Category")
        for ajet in jets
            # get their b-tag value
            btag = getParameters(getParticleID(pidh, ajet, ilcfi))[ibtag]
            ctag = getParameters(getParticleID(pidh, ajet, ilcfi))[ictag]
            otag = getParameters(getParticleID(pidh, ajet, ilcfi))[iotag]
            cat = getParameters(getParticleID(pidh, ajet, ilcfi))[icat]
            println(btag, "\t", ctag, "\t", otag, "\t", cat)
        end
        break
    end
end

## Putting it to use: Developing new functionality

In [ ]:
function prettyPrintMCP(mcp)
    parentPDGs = Int[]
    pdg = getPDG(mcp)
    if 90 < pdg < 99 
        name = "internal"
    elseif abs(pdg) > 20000
        name = "unknown"
    else
        # some pdg values don't have a nice unicode representation
        name = abs(pdg) ∈ (12, 14, 16, 130, 310, 321) ? Particle(pdg).name : to_latex(Particle(pdg).latex)
        parents = getParents(mcp)
        for p in parents
            push!(parentPDGs, getPDG(p))
        end            
    end
    @printf("%d %s %s %.2f %s\n",
        pdg,
        name,
        join(parentPDGs, " "),
        norm(getMomentum(mcp)),
        getGeneratorStatus(mcp)
    )
end

In [ ]:
import CxxWrap
mutable struct VertexNode
    parent::CxxWrap.CxxWrapCore.CxxPtr{LCIO.MCParticle}
    position
    tracks::Vector{CxxWrap.CxxWrapCore.CxxPtr{LCIO.MCParticle}}
    neutrals::Vector{CxxWrap.CxxWrapCore.CxxPtr{LCIO.MCParticle}}
    children::Vector{VertexNode}
end
VertexNode(parent, pos) = VertexNode(parent, pos, [], [], [])

# offspring is the pointer to the vertex tree
function getMCPLineage(mcp, vtx) 
    pdg = abs(getPDG(mcp))
    if pdg ∈ (12, 14, 16, 22, 111, 130, 2112)
        push!(vtx.neutrals, mcp)
        return vtx
    elseif pdg ∈ (11, 13, 211, 321, 2212)
        push!(vtx.tracks, mcp)
        return vtx
    end
    thisVtx = vtx
    endpoint = getEndpoint(mcp)
    if norm(endpoint-vtx.position) > 0.005 # 5 μm minimum flight distance to make a new vtx
        thisVtx = VertexNode(mcp, endpoint)
        push!(vtx.children, thisVtx)
    end
    daug = getDaughters(mcp)
    for d in daug
        getMCPLineage(d, thisVtx)
    end
    vtx
end

In [ ]:
function prettyPrintVtx(vtx::VertexNode, level::Int64=0)
    print(repeat("\t", level))
    @printf("x: %.2f y: %.2f z: %.2f    ", vtx.position[1], vtx.position[3], vtx.position[3])
    prettyPrintMCP(vtx.parent)
    print(repeat("\t", level))
    println("nTracks: ", length(vtx.tracks))
    for t in vtx.tracks
        print(repeat("\t", level+1))
        prettyPrintMCP(t)
    end
    print(repeat("\t", level))
    println("nNeutrals: ", length(vtx.neutrals))
    for n in vtx.neutrals
        print(repeat("\t", level+1))
        prettyPrintMCP(n)
    end
    println(repeat("\t", level), "Children: ", length(vtx.children))
    for c in vtx.children
        prettyPrintVtx(c, level+1)
    end
end

In [ ]:
function computeVertexList(fileList)    
    totalEvents = 0
    for FILENAME in fileList
        LCIO.open(FILENAME) do reader
            for (iEvent, event) in enumerate(reader)
                vtxList = Vector{VertexNode}()
                totalEvents += 1
                if totalEvents > 1
                    break
                end
                vtxCollection = getCollection(event, "BuildUpVertex") # no parameters on the collection
                vtxPartCollection = getCollection(event, "BuildUpVertex_RP") # no parameters on the collection
                mcpartCollection = getCollection(event, "MCParticlesSkimmed")
                reco2truthLink = getCollection(event, "RecoMCTruthLink")
                relnav = LCIO.LCRelationNavigator(reco2truthLink)
                for mcp in mcpartCollection
                    if getPDG(mcp) != 92
                        continue
                    end
                    # the endpoint of the fragmentation state is the IP
                    origin = getEndpoint(mcp)
                    vtx = getMCPLineage(mcp, VertexNode(mcp, origin))
                    push!(vtxList, vtx)
                end
                for vtxPart in vtxPartCollection
                    println("Vtx: ", getType(vtxPart), "\t", getMass(vtxPart), "\t", length(getParticles(vtxPart)), "children")
                    matchedMCVertices = VertexNode[]
                    for (pidx, p) in enumerate(getParticles(vtxPart))
                        if getCharge(p) == 0
                            continue
                        end
                        print("\t", pidx, ". child: ", getType(p))
                        mcParts = getRelatedToObjects(relnav, p)
                        # skip bad matches
                        if length(mcParts) != 1
                            continue
                        end
                        mcp = mcParts[1]
                        println(" match: ", getPDG(mcp))
                        for v in vtxList
                            isFound, vtx = findMCP_in_VtxChain(mcp, v)
                            if isFound
#                                 println("found part:", getType(p), " at ")
#                                 println(getEndVertex(vtxPart))
#                                 prettyPrintVtx(v)
                                push!(matchedMCVertices, v)
                                break
                            end
                        end
                    end
                    if isempty(matchedMCVertices)
                        println("No maches found!")
                        continue
                    end
                    # if not all tracks are from the same vertex, print a warning
                    println([norm(v.position) for v in matchedMCVertices])
#                     if !all(matchedMCVertices .=== matchedMCVertices[1])
#                         println("Reco vtx was matched to multiple MC vertices")
#                     end
                end
            end
        end
    end
end

In [ ]:
flavorTagDir = readdir("/pnfs/desy.de/ilc/prod/ilc/mc-2020/ild/dst-merged/250-SetA/flavortag/ILD_l5_o1_v02/v02-02", join=true)
bbFiles = filter(name -> occursin(r"h_bb",name), flavorTagDir)
bb_vtxList = computeVertexList(bbFiles);

In [ ]:
LCIO.open(bbFiles[1]) do reader
    for event in reader
        @printf("%-6s %10s %-15s%-10s%-16s%-3s\n", "Index", "PDG", "name", "parents", "momentum (GeV)", "generatorStatus")
        for (idx, mcp) in enumerate(getCollection(event, "MCParticlesSkimmed"))
            parentPDGs = Int[]
            pdg = getPDG(mcp)
            if 90 < pdg < 99 
                name = "internal"
            elseif abs(pdg) > 20000
                name = "unknown"
            else
                name = Particle(pdg).name
                parents = getParents(mcp)
                for p in parents
                    push!(parentPDGs, getPDG(p))
                end            
            end
            @printf("%6s %10s %-15s%-10s%13.3f%7s\n",
                idx,
                pdg,
                name,
                join(parentPDGs, " "),
                norm(getMomentum(mcp)),
                getGeneratorStatus(mcp)
            )
        end
        println(repeat("-", 100))
        break
    end
end

In [ ]:
function printMCVertex(FILENAME)
    totalEvents = 0
    LCIO.open(FILENAME) do reader
        for (iEvent, event) in enumerate(reader)
            vtxList = Vector{VertexNode}()
            totalEvents += 1
            vtxCollection = getCollection(event, "BuildUpVertex") # no parameters on the collection
            vtxPartCollection = getCollection(event, "BuildUpVertex_RP") # no parameters on the collection
            mcpartCollection = getCollection(event, "MCParticlesSkimmed")
            reco2truthLink = getCollection(event, "RecoMCTruthLink")
            relnav = LCIO.LCRelationNavigator(reco2truthLink)
            for mcp in mcpartCollection
                if getPDG(mcp) != 92
                    continue
                end
                # the endpoint of the fragmentation state is the IP
                origin = getEndpoint(mcp)
                vtx = getMCPLineage(mcp, VertexNode(mcp, origin))
                prettyPrintVtx(vtx)
            end
            break
        end
    end
end

In [ ]:
printMCVertex(bbFiles[1])